In [1]:
import numpy as np
import pandas as pd
import cv2

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [3]:
import tensorflow as tf

In [22]:
df = pd.read_csv("/content/sample_data/mnist_train_small.csv", header=None)
X = df.iloc[:,1:].values
y = df.iloc[:,0].values
print(X.shape, y.shape)

(20000, 784) (20000,)


In [24]:
df[0].value_counts()

1    2243
7    2126
6    2039
9    2023
3    2021
2    1989
0    1962
4    1924
8    1912
5    1761
Name: 0, dtype: int64

In [45]:
def output_neourn(X, w, b):
  return tf.matmul(X, w) + b

def activation(ynet):
  return tf.nn.softmax(ynet)

def loss(y_true, logits):
  return tf.reduce_mean(tf.losses.categorical_crossentropy(tf.one_hot(y_true,10), logits))

784

# **Perceptron**

In [103]:
%%time
sc=StandardScaler()
X_new=sc.fit_transform(X)
tf.random.set_seed(3)
w=tf.Variable(tf.random.truncated_normal(shape=[784,10],dtype='double'))
b=tf.Variable(tf.random.truncated_normal(shape=[10],dtype='double'))
lr = 1
optimizer=tf.optimizers.SGD(learning_rate=lr)
for epoch in range(5101):
  with tf.GradientTape() as tape:
    logits=activation(output_neourn(X_new,w,b))
    if epoch%100==0: 
      score = accuracy_score(y,tf.argmax(logits,1))
      print(f"epoch: {epoch},\t learning_rate: {lr:.5%},\t score: {score:.5%}")
      lr =  1 - score if lr > 0.0 else 0.0001
    optimizer.apply_gradients(zip(tape.gradient(loss(y,logits),[w,b]),[w,b]))

epoch: 0,           learning_rate: 1,      score: 0.1165
epoch: 100,           learning_rate: 0.8835,      score: 0.8355
epoch: 200,           learning_rate: 0.16449999999999998,      score: 0.8741
epoch: 300,           learning_rate: 0.1259,      score: 0.89005
epoch: 400,           learning_rate: 0.10994999999999999,      score: 0.90115
epoch: 500,           learning_rate: 0.09885,      score: 0.9083
epoch: 600,           learning_rate: 0.0917,      score: 0.9128
epoch: 700,           learning_rate: 0.08720000000000006,      score: 0.91675
epoch: 800,           learning_rate: 0.08325000000000005,      score: 0.92105
epoch: 900,           learning_rate: 0.07894999999999996,      score: 0.92485
epoch: 1000,           learning_rate: 0.07515000000000005,      score: 0.9274
epoch: 1100,           learning_rate: 0.0726,      score: 0.92975
epoch: 1200,           learning_rate: 0.07025000000000003,      score: 0.93205
epoch: 1300,           learning_rate: 0.06794999999999995,      score: 0.

In [100]:
df_test=pd.read_csv('sample_data/mnist_test.csv',header=None)
X_test=df.iloc[:,1:].values
y_test=df.iloc[:,0].values
X_test_new=sc.transform(X_test)
ynet=output_neourn(X_test_new,w,b)
logits=activation(ynet)
ls=loss(y_test,logits)
yhat=tf.argmax(logits,1)
print('score:',accuracy_score(y_test,yhat))

score: 0.9274


In [102]:
gray=cv2.imread('sample_data/0.jpeg',0)
gray=gray.reshape(1,-1)
gray=sc.transform(gray)
ynet=output_neourn(gray,w,b)
logits=activation(ynet)
pred=tf.argmax(logits,1)
print(pred)

tf.Tensor([0], shape=(1,), dtype=int64)


**Hidden Layer**

In [118]:
%%time
def activation_hidden(ynet):
  return tf.nn.tanh(ynet)

wh1=tf.Variable(tf.random.truncated_normal(shape=[784,200],dtype=tf.double))
bh1=tf.Variable(tf.random.truncated_normal(shape=[200],dtype=tf.double))

wh2=tf.Variable(tf.random.truncated_normal(shape=[200,100],dtype=tf.double))
bh2=tf.Variable(tf.random.truncated_normal(shape=[100],dtype=tf.double))

wo=tf.Variable(tf.random.truncated_normal(shape=[100,10],dtype=tf.double))
bo=tf.Variable(tf.random.truncated_normal(shape=[10],dtype=tf.double))
lr = 1
optimizer=tf.optimizers.SGD(learning_rate=lr)
for epoch in range(1000):  
  with tf.GradientTape() as tape:
    ynet1=output_neourn(X_new,wh1,bh1)
    ynet2=output_neourn(activation_hidden(ynet1),wh2,bh2)
    logits=activation(output_neourn(activation_hidden(ynet2),wo,bo))
    if epoch%10 == 0:
      score = accuracy_score(y,tf.argmax(logits,1))
      print(f"epoch: {epoch} \t learning_rate:{lr:.5%} \t Score: {score:.5%}")
      lr = 1- score if lr > 0 else 0.001
    wb = [wh1,bh1,wh2,bh2,wo,bo]
    optimizer.apply_gradients(zip(tape.gradient(loss(y,logits),wb),wb))


epoch: 0 	 learning_rate:100.00000% 	 Score: 10.71500%
epoch: 10 	 learning_rate:89.28500% 	 Score: 52.73000%
epoch: 20 	 learning_rate:47.27000% 	 Score: 67.63500%
epoch: 30 	 learning_rate:32.36500% 	 Score: 74.62000%
epoch: 40 	 learning_rate:25.38000% 	 Score: 79.41500%
epoch: 50 	 learning_rate:20.58500% 	 Score: 82.36500%
epoch: 60 	 learning_rate:17.63500% 	 Score: 84.62500%
epoch: 70 	 learning_rate:15.37500% 	 Score: 86.31500%
epoch: 80 	 learning_rate:13.68500% 	 Score: 87.51500%
epoch: 90 	 learning_rate:12.48500% 	 Score: 88.53000%
epoch: 100 	 learning_rate:11.47000% 	 Score: 89.43000%
epoch: 110 	 learning_rate:10.57000% 	 Score: 90.21500%
epoch: 120 	 learning_rate:9.78500% 	 Score: 90.89500%
epoch: 130 	 learning_rate:9.10500% 	 Score: 91.49500%
epoch: 140 	 learning_rate:8.50500% 	 Score: 91.99500%
epoch: 150 	 learning_rate:8.00500% 	 Score: 92.39000%
epoch: 160 	 learning_rate:7.61000% 	 Score: 92.75000%
epoch: 170 	 learning_rate:7.25000% 	 Score: 93.10000%
epoch: 1

In [127]:
gray=cv2.imread('sample_data/0.jpeg',0)
if gray is not None:
  gray=gray.reshape(1,-1)
  gray=sc.transform(gray)
  ynet=output_neourn(gray,wh1,bh1)
  ynet=output_neourn(activation_hidden(ynet),wh2,bh2)
  ynet=output_neourn(activation_hidden(ynet),wo,bo)
  logits=activation(ynet)
  pred=tf.argmax(logits,1)
  print(pred)

tf.Tensor([5], shape=(1,), dtype=int64)
